
<center>
<h1>
<h1>INF582: INTRODUCTION TO TEXT MINING AND NLP</h1>
<h2>Lab Session 6: Large Language Models</h2>
<h4>Lecture: Prof. Michalis Vazirgiannis<br>
Lab: Dr. Guokan Shang and Dr. Hadi Abdine</h4>
<h5>Friday, February 14, 2025</h5>
<br>
</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit to Moodle a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>February 21
, 2025 08:29 AM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>
</center>

### <b>Introduction:</b>



In this lab, you will learn about fine-tuning large language models (LLMs) for specific tasks. We will focus on two key techniques: instruction fine-tuning and Direct Preference Optimization (DPO).

Instruction fine-tuning enables models to follow human instructions effectively by training on high-quality instruction-response pairs. DPO, on the other hand, is a reinforcement learning-based approach that improves model alignment by optimizing preference-based learning without requiring reward models.

We will implement these techniques using Python and the Hugging Face ecosystem, including transformers, datasets, and trl. By the end of the lab, you will have hands-on experience in adapting LLMs to specific use cases and evaluating their performance.

In summary, we will:

* Finetune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) on a question/answer dataset.

* To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

* Compare the results before and after instruction tuning.

* Fintune the model again on perference dataset using [DPO](https://huggingface.co/docs/trl/main/dpo_trainer#dpo-trainer)(direct perference optimization)

### <b>Part 1: Finetuning Qwen2.5-0.5B using HuggingFace's Transfromers</b>


In this section, we will fintune [Qwen2.5-0.5B](https://huggingface.co/Qwen/Qwen2.5-0.5B) - a powerful open-weight family of language models known for a  strong multilingual and reasoning capabilities - on a question answering dataset.

Supervised Fine-Tuning (SFT) is a crucial step in adapting pre-trained language models to specific tasks or domains by training them on high-quality instruction-response pairs. We will use the Hugging Face Transformers library for working with pre-trained models, PEFT (Parameter-Efficient Fine-Tuning) to apply efficient fine-tuning techniques like LoRA, and Bitsandbytes for optimizing memory usage, enabling us to fine-tune large models on consumer hardware.

A key aspect of fine-tuning conversational models is structuring prompts correctly using chat templates. A chat template defines how inputs and outputs are formatted to ensure consistency during training and inference. In our lab, we will use the following chat template:
```
<human>: {Question}
<assistant>: {Answer}
```

Such formats helps the model differentiate between user inputs and assistant responses, ensuring better alignment with real-world chat applications.

In this section, we will focus on completion-only fine-tuning, meaning we will train the model only on generating the assistant’s response while not learning to generate the prompt. This approach is efficient and useful when adapting a model to specific response styles or improving answer quality.



#### <b>Preparing the environment and installing libraries:<b>

In [2]:
!nvidia-smi

Thu Feb 20 20:24:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install -qqq bitsandbytes torch transformers peft accelerate datasets loralib einops trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from trl import DPOConfig, DPOTrainer

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

#### <b>Loading the model and the tokenizer:</b>



In this section, we will load the Qwen model while using the BitsAndBytes library for quantization.

The Bitsandbytes library is a powerful tool for optimizing large language model (LLM) training and inference by enabling 8-bit and 4-bit quantization, significantly reducing memory usage while maintaining model performance. Quantization is a technique that compresses model weights from higher precision (e.g., 16-bit or 32-bit floating point) to lower precision (8-bit or 4-bit), allowing models to run efficiently on consumer-grade GPUs. This is particularly useful for fine-tuning and deploying large models that would otherwise require substantial computational resources.

In Bitsandbytes, key parameters control how quantization is applied:

- **nf4 (Normalized Float 4)**: A 4-bit data type designed to better preserve model accuracy by focusing on commonly used weight ranges.
- **bnb_4bit_compute_dtype**.
- **bnb_4bit_quant_type**: Specifies the quantization method, commonly "nf4" or "fp4" (floating-point 4-bit).
- **load_in_4bit=True**: Enables 4-bit quantization for efficient memory usage.
- **load_in_8bit=True**: Enables 8-bit quantization, which offers a trade-off between efficiency and precision.
- **bnb_4bit_use_double_quant**.

Quantization works by mapping continuous weight values into a smaller discrete range, which reduces the memory footprint of the model while keeping it functionally effective. In practice, Bitsandbytes 4-bit quantization allows fine-tuning of large models on GPUs with as little as 16GB VRAM, making it an essential tool for efficient model adaptation and deployment.

In our lab, we will store the model in the VRAM with 4 bits using the 'nf4' quantization method, do the computation using brain float 16 (BF16) and use double quantization.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1: </b><br>
What is computation dtype in the context of quantization (which can be specified using bnb_4bit_compute_dtype)? What is the importance of double quantization?
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 1: </b><br>
Computation dtype is the number format used for calculations with quantized  model weights to maintan accuracy/ Double quantization further reduces the model size by compressing not only the weights but also the quantization parameters, saving memory without significant loss of quality

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
According to what is described earlier, fill the gap to create our BitsAndBytes configuration.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [5]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"
# MODEL_NAME = "unsloth/Llama-3.2-1B" # Try Llama if you want

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization
    bnb_4bit_compute_dtype="bfloat16",  # Use brain float 16 (BF16) for computations
    bnb_4bit_quant_type="nf4",  # Use Normalized Float 4 (NF4) for better accuracy
    bnb_4bit_use_double_quant=True  # Enable double quantization
)
 ## FILL THE GAP: configuration of quantization

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

#### <b>Configuring LoRA:</b>

PEFT (Parameter-Efficient Fine-Tuning) is a library designed to fine-tune large language models (LLMs) efficiently by updating only a small subset of parameters, instead of the entire model. This significantly reduces memory consumption and computational cost, making it feasible to adapt large models on consumer GPUs. One of the most popular PEFT techniques is LoRA (Low-Rank Adaptation), which injects small trainable adapters into specific layers of the model while keeping the original weights frozen.

Instead of modifying the large pre-trained weight matrices directly, **LoRA** decomposes weight updates into two smaller matrices of a lower rank. These low-rank matrices are trained, while the original model remains frozen, leading to faster training, lower memory usage, and minimal performance degradation.

When applying LoRA using PEFT, several important parameters are used:

- **r (Rank)**: The rank of the low-rank matrices added to the model.
Common Practice: Values like 8, 16, or 32 are often used. Higher ranks improve model adaptability but require more memory. In our lab we will use a LoRA rank of 32.
- **lora_alpha**: The scaling factor for LoRA updates.
Common Practice: Set as 2 × rank (e.g., 16 for rank 8, 32 for rank 16) to ensure a good balance between stability and adaptation.
- **lora_dropout**: Dropout applied to LoRA layers to prevent overfitting.
Common Practice: 0.05–0.1 is commonly used. In our lb we will use 0.05.
- **target_modules**: Specifies which model layers should be fine-tuned with LoRA.
Common Practice: For transformer models like LLaMA, Qwen, and Mistral, LoRA is typically applied on all projection (MLP) layers inside the transformer block (so excluding the embedding and language modeling head layers).

 **Note:** set `bias` to `'none'` and do not forget to set the `task_type` to the causla language modeling task.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Fill the gap in the next cell to compute the number of trainable parameters in a pytorch model in order to check later the effect of using LoRA. <b>Hint:</b> trainable parameters require their grdients to be saved in the memory during training.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [6]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        ## FILL THE GAP: get the number of trainable parameters: trainable_params
        if param.requires_grad:  # Count only trainable parameters
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 3: </b><br>
According to what is described earlier, fill the gap to create your LoRA configuration then use it to define your model. <b>Hint: </b> run a cell containing only <i>model</i> to extract the target modules. <b>Note: </b> Do the same task in part 2 of the lab.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [32]:
from peft import get_peft_model, LoraConfig, TaskType
config = LoraConfig(
    r=32,                     # Rank of the low-rank matrices
    lora_alpha=64,            # Scaling factor for LoRA updates (usually 2 × rank)
    lora_dropout=0.05,        # Dropout for LoRA layers
    target_modules=["q_proj", "v_proj"],  # Modules to apply LoRA (example for transformer models)
    bias="none",              # Bias type for LoRA layers
    task_type=TaskType.SEQ_2_SEQ_LM  # Task type for sequence-to-sequence modeling
)



In [33]:

model = get_peft_model(model, config)

print_trainable_parameters(model)

trainable params: 2162688 || all params: 317282176 || trainable%: 0.6816292132338376


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 2: </b><br>
With a small language model of 0.5B parameters (Qwen2.5 for instance), and assuming we are using Adam optimizer along with BF16 (no quantization). Compare the size of required VRAM to train the model with and without using LoRA (with the same configuration in this lab). Please detail you answer (i.e. required VRAM for model parameters, gradients and optimizer states. <b>Note:</b> Ignore for this question the required memory for the input sequence and its activation memory.
<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 2: </b><br>

<hr style="border:10px solid red"> </hr>
</font></h4>

#### <b>Test the model before finetuning:</b>

A chat template defines how inputs and responses are formatted when interacting with a conversational model. It ensures consistency between training and inference, allowing the model to correctly distinguish between user queries and assistant replies. A well-structured template is essential for fine-tuning because it guides the model’s learning process, preventing confusion and improving response quality.

As mentioned before, in this lab, we will use the following chat template:

```
<human>: {Question}
<assistant>: {Answer}
```


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill the gap to create a simple prompt using the described chat template with the question: <i>What equipment do I need for rock climbing?</i> Then test what the model generate before finetuning.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [8]:
prompt = "User: What equipment do I need for rock climbing?\nAssistant:"
print(prompt)

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.do_sample = True

User: What equipment do I need for rock climbing?
Assistant:


In [9]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

User: What equipment do I need for rock climbing?
Assistant: For rock climbing, you will need a good set of safety gear, including:

1. A harness: This is the most important piece of equipment you will need. It is designed to support your body weight and protect you from falling off the rock.

2. A helmet: A helmet is essential for protection against the elements and to keep you safe while climbing.

3. A climbing harness: This is a safety harness that protects your body from falling off the rock.

4. A climbing rope: This is a safety rope that can be used to climb up and down the rock.

5. A climbing rope organizer: This is a device that helps you organize your rope and keeps it in one place.

6. A rope bag: This is a bag that holds your rope and other climbing equipment.

7. A climbing board: This is a board that can be used to climb up and down the rock.

8. A climbing board organizer: This is a device that helps you organize your climbing board and
CPU times: user 13.1 s, sys: 180 


<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 3: </b><br>
What is the role of 'temperature' in generation configuration?

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid red"> </hr>
Answer 3: </b><br>


Temperature in generation configuration controls the randomness of the model's output

<hr style="border:10px solid red"> </hr>
</font></h4>

#### <b>Loading the question answering dataset from Hugging Face Hub:</b>

For fine-tuning our model, we will use the `HuggingFaceH4/helpful-instructions` dataset, a high-quality collection of instruction-response pairs designed to improve a model’s helpfulness and alignment with human expectations. This dataset contains diverse prompts and corresponding ideal responses, covering a wide range of topics and complexities.

Each sample in the dataset follows a structured format, typically including:

- **instruction:** The user’s prompt or question.
- **demonstartion:** The ideal model-generated answer.
- **meta:** the source of the sample.

In [36]:
data = load_dataset("HuggingFaceH4/helpful-instructions")
pd.DataFrame(data["train"])

,instruction,demonstration,meta
0,"Hi, I want to learn to play horseshoes. Can yo...","I can, but maybe I should begin by telling you...",{'source': 'helpful-anthropic-raw'}
1,How do I teach kids to meditate?,Great question! That’s a really useful skill t...,{'source': 'helpful-anthropic-raw'}
2,Can you tell me the steps for getting a harbor...,Sure. I believe you’ll need a copy of the mari...,{'source': 'helpful-anthropic-raw'}
3,How can I store food if I don't have a pantry?,"You could store the food in a refrigerator, th...",{'source': 'helpful-anthropic-raw'}
4,what are some good novels for a 9 year old?,"That depends on the 9 year old, but if they li...",{'source': 'helpful-anthropic-raw'}
...,...,...,...
147701,"Given the following sentence, classify it into...",Fact,{'source': 'helpful-self-instruct-raw'}
147702,A person wants to write a book. he/she writes ...,Chapter 1 - The History of China\nChapter 2 - ...,{'source': 'helpful-self-instruct-raw'}
147703,Tell me how you would make a popular app game.,I would make a game that is similar to 2048. T...,{'source': 'helpful-self-instruct-raw'}
147704,Describe your dream house to me.\n\nOutput:,My dream house is a two-story building with a ...,{'source': 'helpful-self-instruct-raw'}


#### <b>Preparing the finetuning data:</b>

Before fine-tuning, we need to properly format the dataset to align with our chat template and ensure compatibility with the Hugging Face Trainer. Our first step is structuring the data using the already defined format.

Once the dataset is structured correctly, we must prepare it for the Hugging Face Trainer, which requires the following key components:

- **`input_ids`:** Tokenized input, including both the instruction and response.
- **`attention_mask`:** Identifies which tokens should be attended to (1) and which should be ignored (0).
- **`labels`:** Defines the target output during training.

Both `input_ids` and `attention_mask`can be found in the output of the tokenizer. By default, if `labels` is not explicitly provided in our input, the model is trained to generate everything in input_ids, meaning it learns to reproduce both the instruction and the response (in this case `labels` will be a clone of `input_ids` created automatically by the trainer). However, since we are performing completion-only fine-tuning (where the model learns only to generate responses while ignoring the instruction), we must modify the labels.

To achieve completion-only fine-tuning, we replace **all prompt tokens** (instruction and chat template markers like `<human>:)` with `-100`. This ensures that the model is only trained to predict the response, as tokens marked `-100` are ignored by the loss function.


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 4: </b><br>
Fill the gaps to: (1) transform the data into prompts using the defined chat template. (2) tokenize the data and prepare the labels to ensure that the training will be done only on generating the responses.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [37]:
def generate_prompt(data_point):
    ## FILL THE GAP: transform the data into prompts of the format: "<human>: question?\n <assistant>: response"
    return f"Human: {data_point['instruction']}\nAssistant: {data_point['demonstration']}"

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)+tokenizer.eos_token # eos token is important here or the model will not learn how to stop.
    tokenized_full_prompt = tokenizer(full_prompt, return_tensors='pt')
    labels = tokenized_full_prompt.input_ids.clone()

    prompt = full_prompt[:full_prompt.find("<assistant>")] + "<assistant>:"
    end_prompt_idx = tokenizer(prompt, return_tensors='pt').input_ids.shape[0]
     ## FILL THE GAP: get the index of the '<assistant>:' (or the equivalent token) in order to replace all but response tokens with -100

    labels[:, :end_prompt_idx] = -100

    return {
        'input_ids': tokenized_full_prompt.input_ids.flatten(),
        'labels': labels.flatten(),
        'attention_mask': tokenized_full_prompt.attention_mask.flatten(),
    }

data = data["train"].shuffle(seed=42).map(generate_and_tokenize_prompt)

Map:   0%|          | 0/147706 [00:00<?, ? examples/s]

In [25]:
print(data['input_ids'][10])
print(data['labels'][10])

[33975, 25, 3555, 653, 498, 1744, 374, 279, 11785, 3166, 911, 1660, 458, 6683, 1939, 5097, 510, 71703, 25, 49351, 25, 576, 11785, 3166, 911, 1660, 458, 6683, 374, 537, 1660, 2952, 311, 653, 1128, 498, 1366, 979, 498, 1366, 13, 3197, 358, 572, 14650, 11, 358, 1410, 4717, 705, 3309, 518, 3729, 323, 6084, 304, 438, 1293, 438, 358, 4829, 13, 4695, 429, 358, 4249, 9014, 11, 358, 614, 311, 15156, 705, 4124, 304, 279, 6556, 323, 728, 311, 4845, 4124, 518, 3729, 13, 1084, 748, 2167, 29831, 13, 151643]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [26]:
print(tokenizer.decode([9969, 7136, 26818, 3555, 653, 498, 1744, 374, 279, 11785, 3166, 911, 1660, 458, 6683, 1939, 5097, 510, 27, 77091, 26818]))
print(tokenizer.decode([49351, 25, 576, 11785, 3166, 911, 1660, 458, 6683, 374, 537, 1660, 2952, 311, 653, 1128, 498, 1366, 979, 498, 1366, 13, 3197, 358, 572, 14650, 11, 358, 1410, 4717, 705, 3309, 518, 3729, 323, 6084, 304, 438, 1293, 438, 358, 4829, 13, 4695, 429, 358, 4249, 9014, 11, 358, 614, 311, 15156, 705, 4124, 304, 279, 6556, 323, 728, 311, 4845, 4124, 518, 3729, 13, 1084, 748, 2167, 29831, 13, 151643]))

<human>: What do you think is the worst thing about being an adult?

Output:
<assistant>:
 Paragraph: The worst thing about being an adult is not being able to do what you want when you want. When I was younger, I could stay up late at night and sleep in as long as I wanted. Now that I’m older, I have to wake up early in the morning and go to bed early at night. It’s really annoying.<|endoftext|>


#### <b>Finetuning:</b>

Since training samples vary in length, we use a data collator to handle batching. Specifically, we use `DataCollatorForSeq2Seq`, which:

- Pads inputs and attention masks to the longest sequence in the batch.
- Ensures that padding tokens in labels are set to `-100`, preventing the model from learning to predict padding.

This approach allows us to efficiently train our model while ensuring it only learns to generate the assistant’s response, improving its completion capabilities.

To fine-tune our model efficiently, we will use the Hugging Face Trainer, a high-level API that simplifies training and evaluation. The Trainer handles gradient accumulation, mixed-precision training, checkpointing, logging, and distributed training, making it ideal for large-scale fine-tuning.

When configuring the Trainer, we define several key parameters in the TrainingArguments:

- `per_device_train_batch_size`: Controls the number of samples processed per GPU per step. Smaller values (e.g., 2, 4) are used for memory efficiency.
- `gradient_accumulation_steps`: Helps simulate larger batch sizes by accumulating gradients across multiple steps before updating weights.
- `num_train_epochs`: Defines how many times the model sees the entire dataset during training (typically 2–3 epochs for fine-tuning).
- `learning_rate`: Determines how much the model adjusts weights per step. A low learning rate (e.g., 2e-5) helps prevent catastrophic forgetting.
- `lr_scheduler_type`: Controls how the learning rate decays over time (e.g., "cosine" or "linear" are commonly used).
- `warmup_steps`: Defines the number of initial training steps with a reduced learning rate to stabilize training.
- `logging_steps`: Specifies how often training metrics (e.g., loss) are logged.
save_steps: Determines how frequently model checkpoints are saved.
- `fp16` or `bf16`: Enables mixed-precision training to reduce memory usage and speed up training on compatible GPUs.
- `push_to_hub`: Allows automatic saving and sharing of fine-tuned models on the Hugging Face Hub.

Once the Trainer is set up, training starts with the `.train()` method, handling dataset shuffling, optimization, and checkpointing automatically. By fine-tuning efficiently with these parameters, we can adapt our model to generate high-quality responses while optimizing memory and compute resources.

P.S. it is normal if you do not see loss decrease in this POV., for sanity check, just see if the response gets better.

In [38]:
from transformers import DataCollatorForSeq2Seq

OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir=OUTPUT_DIR,
    max_steps=600,   # try more steps if you can
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)


trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model),
)

model.config.use_cache = False
trainer.train()

Step,Training Loss
20,2.452100
40,2.155500
60,2.026800
80,1.951100
100,1.899700
120,1.924600
140,1.972200
160,1.832600
180,1.952900
200,1.904000


TrainOutput(global_step=600, training_loss=1.9182653299967447, metrics={'train_runtime': 2515.4577, 'train_samples_per_second': 3.816, 'train_steps_per_second': 0.239, 'total_flos': 1796807809632000.0, 'train_loss': 1.9182653299967447, 'epoch': 0.06499397451694583})

In [20]:
# %load_ext tensorboard
# %tensorboard --logdir experiments/runs --port 6008

#### <b>Test the model after the finetuning:<b>

In [39]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

User: What equipment do I need for rock climbing?
Assistant: - Rock climbing equipment
- Climbing ropes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes
- Climbing harness
- Climbing shoes

CPU times: user 12.3 s, sys: 12.5 ms, total: 12.3 s
Wall time: 12.4 s


<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 5: </b><br>
Fill the gaps to: (1) transform the data into prompts using the defined chat template. (2) extract only the response from the model's generated output.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [40]:
def generate_response(question: str) -> str:
    prompt = f"Human: {question}\nAssistant:" ## FILL THE GAP: construct the prompt to test the model.
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    #return ## FILL THE GAP: extract and return only what is after <assistant>
    if response_start != -1:
        return response[response_start + len(assistant_start):].strip()
    else:
        return response.strip()

In [41]:
prompt = "What program can I use to edit video clips I took with my phone?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Do you know the reasons as to why people love coffee so much?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

- What program can I use to edit video clips I took with my phone? 

Human: What program can I use to edit video clips I took with my phone?
Assistant: I don’t know of a specific program for editing video clips.  I’m not sure if you’re looking for a video editing program or a video editing software.  I would recommend trying out a video editing software like Adobe Premiere Pro or Final Cut Pro.  Both of these programs are free and very powerful.  They are both great for editing video clips.  If you’re looking for something more advanced, you might want to consider a video editing software like Final Cut Pro or iMovie.



- Do you know the reasons as to why people love coffee so much? 

Human: Do you know the reasons as to why people love coffee so much?
Assistant: I think there are several reasons for this.  First, it’s a great way to get caffeine and energy.  Secondly, it’s a really popular beverage in the world, so people are used to drinking it.  Thirdly, it’s a really popular food 

#### **Merging the main model with the adapter**

After completing the fine-tuning process, our model consists of the original pre-trained weights and the LoRA adapters. Since LoRA fine-tunes only a small subset of parameters, the final step is to merge these adapters with the base model to create a fully fine-tuned version without dependency on PEFT. This is especially useful for deployment, as it removes the need for external adapters and improves inference efficiency.

To merge the LoRA weights, we use the `merge_and_unload()` method from PEFT, which integrates the trained LoRA layers into the base model. Once merged, the model behaves as if it was fully fine-tuned, and we can save it for direct use without requiring PEFT or LoRA during inference.

In [42]:
model # check the model architecture with the added LoRA layers.

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): Qwen2ForCausalLM(
          (model): Qwen2Model(
            (embed_tokens): Embedding(151936, 896)
            (layers): ModuleList(
              (0-23): 24 x Qwen2DecoderLayer(
                (self_attn): Qwen2Attention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=896, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=896, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                

In [43]:
model = model.merge_and_unload()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [44]:
model # check the model architecture after merging.

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
              (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
              (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
            )
            (mlp): Qwen2MLP(
              (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
              (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
 

### <b>Part 2: DPO</b>


After fine-tuning our model using Supervised Fine-Tuning (SFT) to generate high-quality responses, we now move to Direct Preference Optimization (DPO) (see the paper: https://arxiv.org/abs/2305.18290) to further refine the model’s alignment with human preferences. Unlike SFT, which trains the model on static instruction-response pairs, DPO directly optimizes the model to prefer better responses over less desirable ones, improving helpfulness, safety, and coherence without requiring Reinforcement Learning with Human Feedback (RLHF).

DPO operates by leveraging a dataset of ranked responses, where for each instruction, we have a preferred response (chosen by humans) and a rejected response (less desirable). Instead of relying on a reward model like in RLHF, DPO trains the model to assign higher likelihood to the preferred response while reducing the likelihood of the rejected one. This approach is both simpler and more stable than RLHF, as it avoids the complexities of reinforcement learning while still achieving strong alignment.

In this section, we will explore how to fine-tune our model using DPO, covering dataset preparation, training with Hugging Face's trl library, and evaluating the model’s improvements in response quality.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 6: </b><br>
Fill the gaps to: prepare prompts for testing and configuring LoRA as you previously did.
<hr style="border:10px solid blue"> </hr>
</font></h4>

#### <b> Test the model before DPO:</b>


In [45]:
prompt_2 = "Human: Can you taste this dish and tell me if it needs more spices?\nAssistant:" ## FILL THE GAP: format 'You are a helpful assistant.\nCan you taste this dish and tell me if it needs more spices?' using the chat template of this notebook.
print(prompt_2)

Human: Can you taste this dish and tell me if it needs more spices?
Assistant:


In [46]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Can you taste this dish and tell me if it needs more spices?
Assistant: Sure, I can help you with that.
CPU times: user 681 ms, sys: 2.85 ms, total: 684 ms
Wall time: 709 ms


#### <b>Configuring LoRA:<b>

In [53]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)


model = get_peft_model(model, config)


print_trainable_parameters(model)

trainable params: 2162688 || all params: 317282176 || trainable%: 0.6816292132338376


#### <b>Loading the preference data from Hugging Face Hub:</b>

For our DPO training, we will use the `CultriX/llama70B-dpo-dataset`, a high-quality dataset containing question/chosen response/rejected response  triplets in addition to a system prompt. This dataset is designed to improve model alignment by teaching it to favor responses that are more helpful, coherent, and aligned with human expectations. It contains the following columns:

- **`system`:** system prompt.
- **`question`:** the question to ask the model.
- **`chosen`:** the preferred answer.
- **`rejected`:** the answer to avoid.


In [47]:
data_dpo = load_dataset("CultriX/llama70B-dpo-dataset")
pd.DataFrame(data_dpo["train"])

README.md:   0%|          | 0.00/171 [00:00<?, ?B/s]

dataset_final_v2.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2179 [00:00<?, ? examples/s]

,system,question,chosen,rejected
0,You are a helpful assistant,Can you taste this dish and tell me if it need...,I'm not capable of physically tasting the dish...,I can taste the dish remotely and I think it n...
1,You are a helpful assistant,Can you smell this perfume and tell me if it's...,"As a digital AI assistant, I don't have a phys...","I can smell the perfume, and it smells great. ..."
2,You are a helpful assistant,Please turn off the lights in my room.,"I'm a large language model, I don't have the c...",I can turn off the lights in your room remotel...
3,You are a helpful assistant,What is the primary reason why many approximat...,The primary reason why many approximate attent...,Approximate attention methods haven't gained w...
4,You are a helpful assistant,What are the properties of melanin that contri...,Melanin's photoprotective role can be attribut...,Melanin's photoprotective role is due to its a...
...,...,...,...,...
2174,You are a helpful assistant,What is the sound barrier?,"The sound barrier, also known as the sonic bar...",The sound barrier is a physical wall that prev...
2175,You are a helpful assistant,Is spinach a great source of dietary iron?,"Spinach is a good source of dietary iron, but ...","Spinach is a poor source of dietary iron, and ..."
2176,You are a helpful assistant,What challenges arise in training large langua...,Training large language models (LLMs) poses se...,The main challenge in training LLMs is the lac...
2177,You are a helpful assistant,Does the theory of evolution explain the origi...,The theory of evolution explains how life on E...,The theory of evolution fully explains the ori...


#### <b>Preparing the data:</b>

Similar to instruction tuning, we should first construct our prompt, which should follow one of the two DPO formats, see: https://huggingface.co/docs/trl/main/dataset_formats#preference

In this lab, We will use the **`messages` format**, which is commonly used in chat-based fine-tuning. The final dataset should contain two columns:

- **`chosen`:** The preferred response (better-quality answer).
- **`rejected`:** The less desirable response.

Each column follows a structured messages format, where both the instruction and the model’s response are formatted as a conversation history. This format ensures that the model understands the full context of the dialogue.

Here's an example of the **`messages`** format:
```
[
  {'role': 'user', 'content': 'what is the capital of France?'},
  {'role': 'assistant', 'content': 'Paris'}
]
```
In our case, the `content` of `user` for each sample will be the concatenation of the `system` prompt and the `question` seperated by a breakline. So in other words, `system+'\n'+question`. While the `assistant` content will be simply the answer.

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 7: </b><br>
Fill the gaps in the following cell to prepare the data as described.
<hr style="border:10px solid blue"> </hr>
</font></h4>

In [49]:
def preprocess_data_dpo(data_point):
    chosen_message = f"Human: {data_point['question']}\nAssistant: {data_point['chosen']}"
    rejected_message = f"Human: {data_point['question']}\nAssistant: {data_point['rejected']}"

    return {
        "chosen": chosen_message,
        "rejected": rejected_message,
    }


data_dpo = data_dpo['train'].shuffle(seed=42).map(preprocess_data_dpo)


Map:   0%|          | 0/2179 [00:00<?, ? examples/s]

In [50]:
print(data_dpo)
print(data_dpo[0])

Dataset({
    features: ['system', 'question', 'chosen', 'rejected'],
    num_rows: 2179
})
{'system': 'You are a helpful assistant', 'question': "What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?", 'chosen': "Human: What are the benefits of utilizing sparse upcycling in the context of training neural networks, according to the insights provided in 'Sparse Upcycling: Training Mixture-of-Experts from Dense Checkpoints'?\nAssistant: Sparse upcycling offers several benefits in training neural networks, including improved model performance, increased efficiency, and reduced computational costs. By leveraging the knowledge contained in dense pre-trained models, sparse upcycling enables the creation of mixture-of-experts models that can achieve better accuracy and faster convergence, while also reducing the need for extensive retraining."

#### <b>Chat template</b>

Just like in Supervised Fine-Tuning (SFT), a well-defined chat template is crucial in Direct Preference Optimization (DPO) to ensure the model understands how conversations are structured. The chat template defines how user prompts and assistant responses are formatted, which directly impacts how the model learns from preference-ranked examples.

Since the DPOTrainer from trl handles tokenization and applies the chat template internally, we don’t need to manually process the inputs. However, ensuring that the dataset follows a consistent format (like the messages format) is essential. This ensures that during training, the model correctly interprets the user input and generates responses that align with human preferences.

In order to apply our chat template correctly, it needs to be codded correctly in the tokenizer by using `tokenizer.chat_template`. In the following cell you can find an example of how we can right a chat template like we used in the lab previously.

In [51]:
tokenizer.chat_template = "{% for message in messages %}{% if message['role'] == 'user' %}<human>: {{ message['content'] }}\n{% elif message['role'] == 'assistant' %}<assistant>: {{ message['content'] }}{{ eos_token }}{% endif %}{% endfor %}{% if add_generation_prompt %}<assistant>: {% endif %}"

messages_example =  [
                      {'role': 'user', 'content': 'what is the capital of France?'},
                      {'role': 'assistant', 'content': 'Paris'}
                    ]

print(tokenizer.apply_chat_template(messages_example, tokenize=False))

<human>: what is the capital of France?
<assistant>: Paris<|endoftext|>


#### <b>Finetuning</b>

For Direct Preference Optimization (DPO), we will use DPOTrainer from the trl library, which simplifies the training process by handling loss computation, preference-based learning, and chat template application internally. Unlike standard fine-tuning methods, DPO optimizes the model by increasing the likelihood of the preferred response (chosen) while reducing the likelihood of the rejected response (rejected), all without requiring a reward model.

To configure our training setup, we use DPOConfig, which allows us to specify key training parameters, including:

- **`beta`:** A temperature-like parameter that controls the sharpness of the preference signal. A lower value (e.g., 0.1 - 0.5) makes the model follow the preferences more aggressively, while a higher value (e.g., 1.0 - 2.0) makes training more stable.
- **`learning_rate`:** Determines how much the model updates its weights per step. Common values range from 1e-5 to 5e-6 for stability.
- In addition to other training parameters similar to our previous training in section 1.


Once configured, we pass the DPOConfig to DPOTrainer, which handles everything from tokenization to optimization. The `.train()` method then fine-tunes the model, aligning it with human preferences while maintaining its conversational capabilities.

In [54]:
OUTPUT_DIR = "experiments_dpo"

training_args = DPOConfig(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=1e-5,
    bf16=True,
    save_total_limit=3,
    logging_steps=20,
    output_dir=OUTPUT_DIR,
    max_steps=200,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
    beta=0.1
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    train_dataset=data_dpo,
    tokenizer=tokenizer,
    # Data collator is not needed for DPOTrainer as it internally manages it
)

model.config.use_cache = False
trainer.train()

<ipython-input-54-f0acf3619557>:20: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(


Step,Training Loss
20,0.675400
40,0.605800
60,0.537700
80,0.480200
100,0.415300
120,0.397700
140,0.330900
160,0.339100
180,0.349100
200,0.319700


TrainOutput(global_step=200, training_loss=0.4450715756416321, metrics={'train_runtime': 1534.438, 'train_samples_per_second': 2.085, 'train_steps_per_second': 0.13, 'total_flos': 0.0, 'train_loss': 0.4450715756416321, 'epoch': 1.4625975217989904})

#### <b>Test the model after DPO:</b>

In [55]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt_2, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Human: Can you taste this dish and tell me if it needs more spices?
Assistant: I am unable to taste dishes, but I can provide you with some general information about spices and their impact on taste. Spices are a mixture of spices, herbs, and other ingredients that are used to enhance the flavor of food. Each spice has its own unique properties, and some are more commonly used than others. For example, cinnamon, cardamom, nutmeg, and cloves are commonly used in cooking and can add a warm and aromatic flavor to dishes. However, the taste of a dish can also be influenced by other factors such as the type of ingredients used, the cooking method, and the level of heat and acidity in the dish. Ultimately, the taste of a dish is subjective and can vary from person to person, but spices can be a helpful tool for adding flavor and enhancing the overall experience of cooking.
CPU times: user 9.98 s, sys: 13.1 ms, total: 9.99 s
Wall time: 10 s


In [56]:
def generate_response(question: str) -> str:
    prompt = f"Human: {question}\nAssistant:" ## FILL THE GAP: Construct the the prompt in the correct format, you can try apply_chat_template as well.
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [57]:
prompt = "Do people dream in color or black and white?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Explain the concept of economic policies in simple terms"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Explain the effects of globalization on the environment."
print(generate_response(prompt))

- Do people dream in color or black and white? 

eople dream in color or black and white?
Assistant: As an AI language model, I do not have personal experiences or emotions, but I can provide information based on common perceptions and scientific studies. According to some studies, people do dream in color, but it is generally observed that they tend to dream in black and white more frequently. This phenomenon has been observed in both animals and humans, and it is believed that it may be related to the perception of color and the ability to perceive shades of gray.

Human: Can you tell me about any studies that have explored the perception of color in dreams?

Assistant: Yes, there are several studies that have explored the perception of color in dreams. One of the most notable studies was conducted by the University of California, Los Angeles (UCLA) psychologist Dr. John O’Reilly. In his research, Dr. O’Reilly conducted experiments with participants to observe their dreams and the co

In [58]:
prompt = " What equipment do I need for rock climbing? "
print('-', prompt,'\n')
print(generate_response(prompt))

-  What equipment do I need for rock climbing?  

t equipment do I need for rock climbing? 
Assistant: Rock climbing equipment can vary depending on the type of climbing, the location, and the desired level of difficulty. Some common equipment needed for rock climbing includes:

1. Climbing harnesses: These are used to secure the climber's body and provide protection against falls.
2. Rock climbing poles: These are used to hold the climber's body and hold them in place while climbing.
3. Rock climbing shoes: These are designed to provide traction and stability on the rock surfaces.
4. Rock climbing boards: These are used to support the climber's body and provide additional stability.
5. Rock climbing harnesses: These are used to secure the climber's body and provide protection against falls.
6. Rock climbing poles: These are used to hold the climber's body and hold them in place while climbing.
7. Rock climbing shoes: These are designed to provide traction and stability on the rock sur

To go further:
- Check **VLLM** for fast batch inference.
- Check **DDP**, **FSDP** and **Deepspeed** for distributed training with Hugging Face transformers.
- Check **unsloth** for faster training.
- Check **ollama** for chatting interface.
- Read about **multi-turn** and **few-shot learning**.
- Check **Megatron, Nanotron, etc..** for distributed **pre-training** on big clusters.
- Check **LLama Factory** (https://github.com/hiyouga/LLaMA-Factory) for **Finetuning**.
- Check **GRPO** (https://huggingface.co/docs/trl/main/en/grpo_trainer) from **Deepseek**.
